In [1]:
!pip install -U datasets
!pip install -U transformers
!pip install torchaudio
!pip install hazm
!pip install librosa
!pip install jiwer
!pip install num2fawords

  Using cached numpy-1.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.2 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.2 which is incompatible.


In [1]:
import pandas as pd
import tqdm
import os

In [2]:
!wget -O female.zip "https://www.dropbox.com/s/4t6mep8mo4yf81f/female.zip?dl=0"
!wget -O male.zip "https://www.dropbox.com/s/xfi3hi927yxixa9/male.zip?dl=0"
!wget https://github.com/pariajm/sharif-emotional-speech-dataset/raw/master/shemo.json
df=pd.read_json('shemo.json').T
!unzip '*.zip' -d combined
!mv /content/combined/male/*.wav /content/combined/
!mv /content/combined/female/*.wav /content/combined/
!rm -r /content/combined/male/ /content/combined/female/
os.listdir('/content/combined/')[0]
df.index=df.index.map(lambda x : '/content/combined/'+str(x)+'.wav')
files=os.listdir('/content/combined/')
path_list=[]
transcript_list=[]
for file in tqdm.tqdm(files,total=len(files)):
  try:
    address=f'/content/combined/{file}'
    transcript=df.loc[df.index==f'/content/combined/{file}','transcript'].item()
    path_list.append(address)
    transcript_list.append(transcript)
  except ValueError:
    pass
df=pd.DataFrame(data={'path':path_list,'transcript':transcript_list})
df.head()

--2023-09-07 12:47:22--  https://www.dropbox.com/s/4t6mep8mo4yf81f/female.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/4t6mep8mo4yf81f/female.zip [following]
--2023-09-07 12:47:23--  https://www.dropbox.com/s/raw/4t6mep8mo4yf81f/female.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5f791165ce691452682f91ab69.dl.dropboxusercontent.com/cd/0/inline/CDQOAwMMFqrI5T3KjlX45UWrPTzvu3Tf-MKXm_6tikCuE-4UObF4x39fox5t0pc1GmusUa2xaRNu3Q7ccGBUwUiXGNQV_dg8dpOXXi4TveJp3-IeLImqqRlsDZ5uLaZIZYnyAUcm3yVgO9IG7R5zwDdJ/file# [following]
--2023-09-07 12:47:23--  https://uc5f791165ce691452682f91ab69.dl.dropboxusercontent.com/cd/0/inline/CDQOAwMMFqrI5T3KjlX45UWrPTzvu3Tf-MKXm_6tikCuE-4UObF4x39fox5t0pc1GmusUa2xaRNu3Q7ccGBUwUiXGNQV_dg8dpOX

100%|██████████| 3000/3000 [00:01<00:00, 2435.02it/s]


,path,transcript
0,/content/combined/M43A04.wav,تاتاره دیگه این وسط چی کارست؟
1,/content/combined/F07A51.wav,من نمی‎فهمم تو چرا نمی‌تونی پیشنهادشو قبول کنی؟
2,/content/combined/F22H03.wav,تو داری منو لوس می‌کنی پیتر، انگار واقعاً هنرم...
3,/content/combined/F22S08.wav,چه جوری می‌تونم برگردم سمتش درصورتی‌که اون، او...
4,/content/combined/F10W02.wav,مگه چی شده؟


In [3]:
from datasets import load_dataset, load_metric

import numpy as np

import hazm
from num2fawords import words, ordinal_words
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import string
import six
import re
import glob

In [4]:
_normalizer = hazm.Normalizer()

chars_to_ignore = [
    ",", "?", ".", "!", "-", ";", ":", '""', "%", "'", '"', "�",
    "#", "!", "؟", "?", "«", "»", "،", "(", ")", "؛", "'ٔ", "٬",'ٔ', ",", "?",
    ".", "!", "-", ";", ":",'"',"“", "%", "‘", "”", "�", "–", "…", "_", "”", '“', '„',
    'ā', 'š',
#     "ء",
]

# In case of farsi
chars_to_ignore = chars_to_ignore + list(string.ascii_lowercase + string.digits)

chars_to_mapping = {
    'ك': 'ک', 'دِ': 'د', 'بِ': 'ب', 'زِ': 'ز', 'ذِ': 'ذ', 'شِ': 'ش', 'سِ': 'س', 'ى': 'ی',
    'ي': 'ی', 'أ': 'ا', 'ؤ': 'و', "ے": "ی", "ۀ": "ه", "ﭘ": "پ", "ﮐ": "ک", "ﯽ": "ی",
    "ﺎ": "ا", "ﺑ": "ب", "ﺘ": "ت", "ﺧ": "خ", "ﺩ": "د", "ﺱ": "س", "ﻀ": "ض", "ﻌ": "ع",
    "ﻟ": "ل", "ﻡ": "م", "ﻢ": "م", "ﻪ": "ه", "ﻮ": "و", 'ﺍ': "ا", 'ة': "ه",
    'ﯾ': "ی", 'ﯿ': "ی", 'ﺒ': "ب", 'ﺖ': "ت", 'ﺪ': "د", 'ﺮ': "ر", 'ﺴ': "س", 'ﺷ': "ش",
    'ﺸ': "ش", 'ﻋ': "ع", 'ﻤ': "م", 'ﻥ': "ن", 'ﻧ': "ن", 'ﻭ': "و", 'ﺭ': "ر", "ﮔ": "گ",

    # "ها": "  ها", "ئ": "ی",
    "۱۴ام": "۱۴ ام",

    "a": " ای ", "b": " بی ", "c": " سی ", "d": " دی ", "e": " ایی ", "f": " اف ",
    "g": " جی ", "h": " اچ ", "i": " آی ", "j": " جی ", "k": " کی ", "l": " ال ",
    "m": " ام ", "n": " ان ", "o": " او ", "p": " پی ", "q": " کیو ", "r": " آر ",
    "s": " اس ", "t": " تی ", "u": " یو ", "v": " وی ", "w": " دبلیو ", "x": " اکس ",
    "y": " وای ", "z": " زد ",
    "\u200c": " ", "\u200d": " ", "\u200e": " ", "\u200f": " ", "\ufeff": " ",
}


def multiple_replace(text, chars_to_mapping):
    pattern = "|".join(map(re.escape, chars_to_mapping.keys()))
    return re.sub(pattern, lambda m: chars_to_mapping[m.group()], str(text))

def remove_special_characters(text, chars_to_ignore_regex):
    text = re.sub(chars_to_ignore_regex, '', text).lower() + " "
    return text

def normalizer(text, chars_to_ignore=chars_to_ignore, chars_to_mapping=chars_to_mapping):
    chars_to_ignore_regex = f"""[{"".join(chars_to_ignore)}]"""
    text = text.lower().strip()

    text = _normalizer.normalize(text)
    text = multiple_replace(text, chars_to_mapping)
    text = remove_special_characters(text, chars_to_ignore_regex)
    text = re.sub(" +", " ", text)
    _text = []
    for word in text.split():
        try:
            word = int(word)
            _text.append(words(word))
        except:
            _text.append(word)

    text = " ".join(_text) + " "
    text = text.strip()

    if not len(text) > 0:
        return None

    return text + " "

In [5]:
print(f"Step 0: {len(df)}")

df["status"] = df["path"].apply(lambda path: True if os.path.exists(path) else None)
df = df.dropna(subset=["path"])
df = df.drop("status", 1)
print(f"Step 1: {len(df)}")

df["transcript"] = df["transcript"].apply(lambda t: normalizer(t))
df = df.dropna(subset=["transcript"])
print(f"Step 2: {len(df)}")

df = df.sample(frac=1)
df = df.reset_index(drop=True)
df.head()

Step 0: 2838
Step 1: 2838


<ipython-input-5-9d2870ecc8ba>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop("status", 1)


Step 2: 2838


,path,transcript
0,/content/combined/M28A43.wav,و مقام و منسب دوکی را از خانواده ما گرفتن
1,/content/combined/M37N12.wav,بعضی وقتا حس می کنم که من بعد از تئودور درمورد...
2,/content/combined/M03A34.wav,داره همه وجود منو تسخیر می کنه داره خودشو به م...
3,/content/combined/F21N14.wav,ببینم شما خیلی وقته اینجا کار می کنید
4,/content/combined/F19N32.wav,حال بشنویم آخرین بخش از سرگذشت پروین اعتصامی را


In [6]:
main_vocab = ["ح", "چ", "ج", "ث", "ت", "پ", "ب", "آ", "ا", "ش", "س", "ژ", "ز", "ر", "ذ", "د", "خ", "ق", "ف", "غ", "ع", "ظ", "ط", "ض", "ص", "ی", "ه", "و", "ن", "م", "ل", "گ", "ک"]
text = " ".join(df["transcript"].values.tolist())
vocab = list(sorted(set(text)))

for v in main_vocab:
    if v not in vocab:
        print("v", v)

print(len(main_vocab), len(vocab))
print(vocab)

33 36
[' ', 'ء', 'آ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی']


In [7]:
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np

idx = np.random.randint(0, len(df))
print(idx)
sample = df.iloc[idx]

path = sample["path"]
print(sample["transcript"], "\n")
speech, sr = torchaudio.load(path)
speech = speech[0].numpy().squeeze()

speech = librosa.resample(np.asarray(speech), sr, 16_000)
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

162
من دویست صفحه دیگه هم خوندم  



In [8]:
train_df, test_df = train_test_split(df[:1000], test_size=0.2, random_state=101)

train_df = train_df[["path", "transcript"]]
train_df = train_df.reset_index(drop=True)

test_df = test_df[["path", "transcript"]]
test_df = test_df.reset_index(drop=True)

print(train_df.shape)
print(test_df.shape)

(800, 2)
(200, 2)


In [9]:
train_df.to_csv("/content/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv("/content/test.csv", sep="\t", encoding="utf-8", index=False)

print(train_df.shape)
print(test_df.shape)

(800, 2)
(200, 2)


In [10]:
common_voice_train = load_dataset("csv", data_files={"train": "/content/train.csv"}, delimiter="\t")["train"]
common_voice_test = load_dataset("csv", data_files={"test": "/content/test.csv"}, delimiter="\t")["test"]

print(common_voice_train)
print(common_voice_test)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['path', 'transcript'],
    num_rows: 800
})
Dataset({
    features: ['path', 'transcript'],
    num_rows: 200
})


In [11]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [12]:
show_random_elements(common_voice_train.remove_columns(["path"]), num_examples=20)

,transcript
0,و دکتر بهشتی در محضر اساتیدی در این زمان درس خواندن که درس رو با بحث درباره سیاست در هم آمیخته بودند
1,کسی که بتواند از آن سیاه چال بگریزد و همه را فریب دهد پس بسیار آسان هم می تواند حکومت مرا منقرض کند
2,اگه یه حرفه احمقانه بزنم نمی خندین
3,بهت که گفتم اگه منو تیکه تیکه هم بکنی حاضر نیستم به مردم بهتون بزنم
4,تو نامه ها و کارت پستال هایی که برای مادرش می فرستاده یه مقدار آدرسو اسم چند نفر هست
5,به سلامت
6,اون موضوع رو به من گفته
7,خیال می کنی من چه مدت درازی می تونم در این قفسه تنگ انتظار بکشم
8,درسته بله
9,پدر من به امر فرمانروای سابق به دار آویخته شد


In [13]:
def normalizer(batch, chars_to_ignore=chars_to_ignore, chars_to_mapping=chars_to_mapping):
    chars_to_ignore_regex = f"""[{"".join(chars_to_ignore)}]"""
    text = batch["transcript"].lower().strip()

    text = _normalizer.normalize(text)
    text = multiple_replace(text, chars_to_mapping)
    text = remove_special_characters(text, chars_to_ignore_regex)
    text = re.sub(" +", " ", text)
    _text = []
    for word in text.split():
        try:
            word = int(word)
            _text.append(words(word))
        except:
            _text.append(word)

    text = " ".join(_text) + " "
    text = text.strip()

    if not len(text) > 0:
        return None

    batch["transcript"] = text

    return batch

In [14]:
print(common_voice_train[0]["transcript"])
print(common_voice_test[0]["transcript"])

آه ازتون ممنونم آقای پوآرو 
اما امیررضا به مادرش قول داده که نشانی از برادر گمشده اش پیدا کنه 


In [15]:
common_voice_train = common_voice_train.map(normalizer, fn_kwargs={"chars_to_ignore": chars_to_ignore, "chars_to_mapping": chars_to_mapping})
common_voice_test = common_voice_test.map(normalizer, fn_kwargs={"chars_to_ignore": chars_to_ignore, "chars_to_mapping": chars_to_mapping})

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [16]:
print(common_voice_train[0]["transcript"])
print(common_voice_test[0]["transcript"])

آه ازتون ممنونم آقای پوآرو
اما امیررضا به مادرش قول داده که نشانی از برادر گمشده اش پیدا کنه


In [17]:
def extract_all_chars(batch):
    all_text = " ".join(batch["transcript"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [18]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [19]:
vocab_list = list(sorted(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0])))
vocab_list = [vocab for vocab in vocab_list if vocab not in [" ", "\u0307"]]
print(len(vocab_list))
print(vocab_list)

34
['آ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی']


In [20]:
vocab_list = list(sorted(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0])))
vocab_list = [vocab for vocab in vocab_list if vocab not in [" ", "\u0307"]]
print(len(vocab_list))
print(vocab_list)

34
['آ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی']


In [21]:
special_vocab = ["<pad>", "<s>", "</s>", "<unk>", "|"]

In [22]:
vocab_dict = {v: k for k, v in enumerate(special_vocab + vocab_list)}
print(len(vocab_dict))
print(vocab_dict)

39
{'<pad>': 0, '<s>': 1, '</s>': 2, '<unk>': 3, '|': 4, 'آ': 5, 'ئ': 6, 'ا': 7, 'ب': 8, 'ت': 9, 'ث': 10, 'ج': 11, 'ح': 12, 'خ': 13, 'د': 14, 'ذ': 15, 'ر': 16, 'ز': 17, 'س': 18, 'ش': 19, 'ص': 20, 'ض': 21, 'ط': 22, 'ظ': 23, 'ع': 24, 'غ': 25, 'ف': 26, 'ق': 27, 'ل': 28, 'م': 29, 'ن': 30, 'ه': 31, 'و': 32, 'پ': 33, 'چ': 34, 'ژ': 35, 'ک': 36, 'گ': 37, 'ی': 38}


In [23]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [25]:
save_dir = "/content/hosseini/wav2vec2-large-xlsr-persian-v1"
model_name_or_path = "m3hrdadfi/wav2vec2-large-xlsr-persian-v3"

In [26]:
from transformers.trainer_utils import get_last_checkpoint

last_checkpoint = None
if os.path.exists(save_dir):
    last_checkpoint = get_last_checkpoint(save_dir)

print(last_checkpoint if last_checkpoint else str(None))

None


In [27]:
from transformers import Wav2Vec2CTCTokenizer

if not os.path.exists(save_dir) and not model_name_or_path:
    print("Load from scratch")
    tokenizer = Wav2Vec2CTCTokenizer(
        "./fa.vocab.json",
        bos_token="<s>",
        eos_token="</s>",
        unk_token="<unk>",
        pad_token="<pad>",
        word_delimiter_token="|",
        do_lower_case=False
    )
else:
    print(f"Load from {model_name_or_path}")
    tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(model_name_or_path)

Load from m3hrdadfi/wav2vec2-large-xlsr-persian-v3


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [28]:
text = "از مهمونداری کنار بکشم"
print(" ".join(tokenizer.tokenize(text)))
print(tokenizer.decode(tokenizer.encode(text)))

ا ز | م ه م و ن د ا ر ی | ک ن ا ر | ب ک ش م
از مهمونداری کنار بکشم


In [29]:
from transformers import Wav2Vec2FeatureExtractor

if not os.path.exists(save_dir) and not model_name_or_path:
    print("Load from scratch")
    feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
else:
    print(f"Load from {model_name_or_path}")
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)

Load from m3hrdadfi/wav2vec2-large-xlsr-persian-v3


In [30]:
from transformers import Wav2Vec2Processor

if not os.path.exists(save_dir) and not model_name_or_path:
    print("Load from scratch")
    processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
else:
    print(f"Load from {model_name_or_path}")
    processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)

Load from m3hrdadfi/wav2vec2-large-xlsr-persian-v3


In [31]:
if len(processor.tokenizer.get_vocab()) == len(processor.tokenizer):
    print(len(processor.tokenizer))

40


In [32]:
if not os.path.exists(save_dir):
    print("Saving ...")
    processor.save_pretrained(save_dir)
    print("Saved!")

Saving ...
Saved!


In [33]:
common_voice_train[0]

{'path': '/content/combined/F21H11.wav',
 'transcript': 'آه ازتون ممنونم آقای پوآرو'}

In [34]:
import torchaudio
import librosa


target_sampling_rate = 16_000

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, target_sampling_rate)


    batch["speech"] = speech_array
    batch["sampling_rate"] = target_sampling_rate
    batch["duration_in_seconds"] = len(batch["speech"]) / target_sampling_rate
    batch["target_text"] = batch["transcript"]
    return batch

In [35]:
common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names, num_proc=4)
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names, num_proc=4)

Map (num_proc=4):   0%|          | 0/800 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

In [36]:
common_voice_train[0].keys()

dict_keys(['speech', 'sampling_rate', 'duration_in_seconds', 'target_text'])

In [37]:
print(common_voice_train[0]["sampling_rate"])
print(common_voice_test[0]["sampling_rate"])

16000
16000


In [38]:
min_duration_in_seconds = 5.0
max_duration_in_seconds = 10.0

def filter_by_max_duration(batch):
    return min_duration_in_seconds <= batch["duration_in_seconds"] <= max_duration_in_seconds

In [39]:
print(f"Split sizes [BEFORE]: {len(common_voice_train)} train and {len(common_voice_test)} validation.")

_common_voice_train = common_voice_train.filter(filter_by_max_duration, num_proc=4)
_common_voice_test = common_voice_test
# _common_voice_test = common_voice_test.filter(filter_by_max_duration, num_proc=4)

print(f"Split sizes [AFTER]: {len(_common_voice_train)} train and {len(_common_voice_test)} validation.")

Split sizes [BEFORE]: 800 train and 200 validation.


Filter (num_proc=4):   0%|          | 0/800 [00:00<?, ? examples/s]

Split sizes [AFTER]: 180 train and 200 validation.


In [40]:
import IPython.display as ipd
import numpy as np
import random

sample = _common_voice_train
rand_int = random.randint(0, len(sample))

print("Target text:", sample[rand_int]["target_text"])
print("Input array shape:", np.asarray(sample[rand_int]["speech"]).shape)
print("Sampling rate:", sample[rand_int]["sampling_rate"])

ipd.Audio(data=np.asarray(sample[rand_int]["speech"]), autoplay=True, rate=16000)

Target text: بعد از شهادت مهندس نتونستم هر وقت خواستم بیام یه بغض سنگین اومد گلومو گرفت
Input array shape: (109780,)
Sampling rate: 16000


In [41]:
import IPython.display as ipd
import numpy as np
import random

sample = _common_voice_train
rand_int = random.randint(0, len(sample))

print("Target text:", sample[rand_int]["target_text"])
print("Input array shape:", np.asarray(sample[rand_int]["speech"]).shape)
print("Sampling rate:", sample[rand_int]["sampling_rate"])

ipd.Audio(data=np.asarray(sample[rand_int]["speech"]), autoplay=True, rate=16000)

Target text: قدم زدن یه دختر تنها تو بیشه زار حتی اگر هم صحیح و سالم باشه سراسر ایراده
Input array shape: (103340,)
Sampling rate: 16000


In [42]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [43]:
_common_voice_train = _common_voice_train.map(prepare_dataset, remove_columns=_common_voice_train.column_names, batch_size=8, num_proc=4, batched=True)
_common_voice_test = _common_voice_test.map(prepare_dataset, remove_columns=_common_voice_test.column_names, batch_size=8, num_proc=4, batched=True)

Map (num_proc=4):   0%|          | 0/180 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

In [44]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [45]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [46]:
wer_metric = load_metric("wer")

<ipython-input-46-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [47]:
import random


def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    if isinstance(label_str, list):
        if isinstance(pred_str, list) and len(pred_str) == len(label_str):
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: "{label_str[index]}"')
                print(f'predicted: "{pred_str[index]}"')

        else:
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: "{label_str[index]}"')
                print(f'predicted: "{pred_str}"')

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [48]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    # "facebook/wav2vec2-large-xlsr-53" if not last_checkpoint else last_checkpoint,
    model_name_or_path if not last_checkpoint else last_checkpoint,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    ctc_zero_infinity=True,
    bos_token_id=processor.tokenizer.bos_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer.get_vocab()))

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [49]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1925: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [50]:
!pip install transformers[torch]
!pip install accelerate>=0.20.1

In [51]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=save_dir,
    group_by_length=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    #gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=0.5,
    fp16=True,
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    learning_rate=1e-4,
    warmup_steps=500,
    save_total_limit=2,
)

In [52]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=_common_voice_train,
    eval_dataset=_common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [53]:
print(f"last_checkpoint: {last_checkpoint}")

last_checkpoint: None


In [54]:
if last_checkpoint:
    print(f"last_checkpoint: {last_checkpoint}")
    train_result = trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    train_result = trainer.train()


metrics = train_result.metrics
max_train_samples = len(_common_voice_train)
metrics["train_samples"] = min(max_train_samples, len(_common_voice_train))

trainer.save_model()

trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
10,2.821400,2.108090,0.436074
20,1.438000,2.076401,0.436505


reference: "وقتی تلفن زدم بهم گفت که رفتی خدمت"
predicted: "سیستم معذهبی ما ی جسم طبیعی که از نوسوش تشکیل شد روح م‌ها در این فضا قرار گرفته و مثل دندانی که در دهان جای میگیرم در جسم ما مستقل شده"
reference: "چی داری می گی پیتر"
predicted: "چی داری میگه پیتر"
reference: "اگه یه بار دیگه مزاحم من بشی می دمت دست پلیس"
predicted: "اگه یه بار دیگه مزاحم مم بشیه میدمی دست پلیس"


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


reference: "هنوز زخم ضرباتش روی سینه ام مونده"
predicted: "هنوز زخمه صرفاتش روی سینا مونده"
reference: "فقط چند کلمه قربان"
predicted: "فقط چند کلم قربم"
reference: "همش می خواستم در لفافه بهت بگم اما خوب شد تو به طور اشتباهی مرگو تجربه کردی"
predicted: "همش میخواستم در لفاف بهت بگم اما خوب شد تو بطور اشتباهی مرگ رو تجربه کردی"


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


***** train metrics *****
  epoch                    =       0.51
  total_flos               = 18153274GF
  train_loss               =     2.2834
  train_runtime            = 0:01:37.61
  train_samples            =        180
  train_samples_per_second =      0.922
  train_steps_per_second   =      0.236


In [55]:
results = {}
metrics = trainer.evaluate()
max_val_samples = len(_common_voice_test)
metrics["eval_samples"] = min(max_val_samples, len(_common_voice_test))
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

reference: "از وضع این بیل لارنر سر در نمی یارم"
predicted: "از وضع این بیلارنش سر در نمیارم"
reference: "شما پادشاه هستین"
predicted: "شما پادشاه هستید"
reference: "اگه پولی که می خوام بدین قول می دم خفه خون بگیرم"
predicted: "اگه پولی که می خوام بریم قور میدم خفه خون بگیرم"
***** eval metrics *****
  epoch                   =       0.51
  eval_loss               =     2.0584
  eval_runtime            = 0:00:15.99
  eval_samples            =        200
  eval_samples_per_second =     12.505
  eval_steps_per_second   =      3.126
  eval_wer                =     0.4365


In [69]:
final_path = "/content/hosseini/wav2vec2-large-xlsr-persian-v1"

In [71]:
model = Wav2Vec2ForCTC.from_pretrained(final_path).to("cuda")
processor = Wav2Vec2Processor.from_pretrained(final_path)

In [72]:
input_dict = processor(_common_voice_test["input_values"][0], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [74]:
common_voice_test_transcription = load_dataset("csv", data_files={"test": "/content/test.csv"}, delimiter="\t")["test"]

In [76]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription["transcript"][0].lower())

Prediction:
اما امیر رضا به مادرش قول دادی که نشانی از برادر گمشدش پیدا کنید

Reference:
اما امیررضا به مادرش قول داده که نشانی از برادر گمشده اش پیدا کنه 
